In [1]:
import pandas as pd
import sqlalchemy as sa
import os
from dotenv import load_dotenv
load_dotenv()

user = os.environ['REDSHIFT_USER']
password = os.environ['REDSHIFT_PASSWORD']
host = os.environ['REDSHIFT_HOST']
port = os.environ['REDSHIFT_PORT']
db = os.environ['REDSHIFT_DB']


In [6]:
def get_sqlalchemy_engine():
    return sa.create_engine(url=f"postgresql://{user}:{password}@{host}:{port}/{db}",
                              pool_size=100,
                              max_overflow=200,
                              client_encoding='utf8',
                              executemany_mode='batch',
                              echo=True)


def get_pandas_df_from_query(q):
    global db_conn
    return pd.read_sql_query(q, db_conn)


def ingest_pandas_df(df, schema, table, mode):
    global db_conn
    r = df.to_sql(name=table, con=db_conn, schema=schema, if_exists=mode, index=False, chunksize=2000, method='multi')
    print(f'Ingested data for {schema}.{table}, Record Count: {len(df)}, Ingested: {r}')


db_conn = get_sqlalchemy_engine()

In [7]:
query = 'select * from public.shipments limit 10;'
df = get_pandas_df_from_query(query)
df

2024-07-24 19:53:10,669 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-07-24 19:53:10,672 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-24 19:53:10,894 INFO sqlalchemy.engine.Engine select current_schema()
2024-07-24 19:53:10,895 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-07-24 19:53:11,239 INFO sqlalchemy.engine.Engine select * from public.shipments limit 10;
2024-07-24 19:53:11,241 INFO sqlalchemy.engine.Engine [raw sql] {}


,id,ext_id,created_at,reference,tracking_number,to_address,order,item_count,value_amount,value_currency,weight_grams,tags,warehouse_code,shipping_service,carrier_ext_id,carrier_name,carrier_ref,notification_email_md5
0,DCUK1/OUT/1517224-15505420873432,d780ae8a-5999-49e0-897b-1ba028390478,2023-06-30T05:44:32Z,DCUK1/OUT/1517224,15505420873432,"{""full_name_md5"":""e5f42ab9472d31ff3fda3e4e4ede...","{""reference"":""Shop_#UK-1511109""}",1,5.0,GBP,36,"[""air up uk"",""air-up-uk-store.myshopify.com""]",DCUK1,DPDND,dpdgbr,DPD UK / Interlink Express,dpd-uk,ccd29d7e5886532aa356f40b1ada2931
1,DCUK1/OUT/1517225-15505420873435,48a732b8-d4e9-4bcc-95a9-c3c7d8aaf958,2023-06-30T05:44:33Z,DCUK1/OUT/1517225,15505420873435,"{""full_name_md5"":""4874266d2c03aa819cc103a6714c...","{""reference"":""Shop_#UK-1511110""}",2,35.0,GBP,649,"[""air up uk"",""air-up-uk-store.myshopify.com""]",DCUK1,DPDND,dpdgbr,DPD UK / Interlink Express,dpd-uk,a083d4eb7c047ee352f496bb90e39c5a
2,DCUK1/OUT/1517227-15505420873441,24a0fec7-65f1-4739-a880-8421c27faa64,2023-06-30T05:44:34Z,DCUK1/OUT/1517227,15505420873441,"{""full_name_md5"":""1ac8215d1987723c4def51f38291...","{""reference"":""Shop_#UK-1511112""}",2,41.0,GBP,773,"[""air up uk"",""air-up-uk-store.myshopify.com""]",DCUK1,DPDND,dpdgbr,DPD UK / Interlink Express,dpd-uk,dd29e9400af67bbf7771d248531f2893
3,DCUK1/OUT/1517228-15505420873444,f9906a5e-1f35-49ec-8f43-47f28adfccd3,2023-06-30T05:44:35Z,DCUK1/OUT/1517228,15505420873444,"{""full_name_md5"":""32e8583c6d416fc4098ef410cb08...","{""reference"":""Shop_#UK-1511113""}",4,46.0,GBP,721,"[""air up uk"",""air-up-uk-store.myshopify.com""]",DCUK1,DPDND,dpdgbr,DPD UK / Interlink Express,dpd-uk,8c8b5b5458f87582aa5ff428429e9b57
4,DEDE1/OUT/1612052-LE360312953DE,4f4c0720-39b3-4572-a984-3ffa8b062e58,2023-06-30T05:50:52Z,DEDE1/OUT/1612052,LE360312953DE,"{""full_name_md5"":""0803b0bc2bab3351b888169b979c...","{""reference"":""Shop_#AT-124964""}",3,23.0,EUR,108,"[""air up at"",""air-up-at-store.myshopify.com""]",DEDE1,DPSTD,depoma,Deutsche Post Mail,ce-de,9166c0dc1df5257d8dd044c963a60dde
5,DEDE1/OUT/1612250-00340434512282310499,bcfc5c4a-dd2c-45ae-a5b6-8ccd9b1e3e74,2023-06-30T05:50:53Z,DEDE1/OUT/1612250,00340434512282310499,"{""full_name_md5"":""7a4951ef1543f31804bb844b9cd6...","{""reference"":""Shop_#DE-2797691""}",3,26.0,EUR,108,"[""air up de"",""ten-ace-gmbh.myshopify.com""]",DEDE1,DPSTD,depoma,Deutsche Post Mail,ce-de,7e6320b8f3fa8581f6888cd1bb1dbc72
6,DEDE1/OUT/1612690-00340434512282310512,a72382e5-5927-4f3e-9572-5ede46a2f00b,2023-06-30T05:50:56Z,DEDE1/OUT/1612690,00340434512282310512,"{""full_name_md5"":""7f27d0ddff050b1c9c9c40454dee...","{""reference"":""Shop_#DE-2798023""}",3,26.0,EUR,108,"[""air up de"",""ten-ace-gmbh.myshopify.com""]",DEDE1,DPSTD,depoma,Deutsche Post Mail,ce-de,d47f7770b04342aee4e0c7ffd6a12eec
7,DEDE1/OUT/1613023-LE360312940DE,76ad33e2-35c8-4482-a004-c19f0db5f0de,2023-06-30T05:50:56Z,DEDE1/OUT/1613023,LE360312940DE,"{""full_name_md5"":""5a086988926bbfc2bd430d614950...","{""reference"":""Shop_#INTL-128711""}",1,20.0,EUR,72,"[""air up intl"",""air-up-intl-store.myshopify.com""]",DEDE1,DPSTD,depoma,Deutsche Post Mail,ce-de,208d90d7da033e5edf62053d0fc451ba
8,DEDE1/OUT/1613160-00340434512282310536,9d910cfa-269c-4e63-9f41-16d38e82ebda,2023-06-30T05:50:57Z,DEDE1/OUT/1613160,00340434512282310536,"{""full_name_md5"":""f67ce4b387670ffc6dd7ad3c0243...","{""reference"":""Shop_#DE-2798389""}",2,20.0,EUR,72,"[""air up de"",""ten-ace-gmbh.myshopify.com""]",DEDE1,DPSTD,depoma,Deutsche Post Mail,ce-de,a33eb4fda5ce80786477c181acacb891
9,DEDE1/OUT/1612436-00340434512282310529,42fcc142-1ce1-4f7d-9453-59267b5cc5f5,2023-06-30T05:50:54Z,DEDE1/OUT/1612436,00340434512282310529,"{""full_name_md5"":""a4653a9d09d63c0efad5b528c094...","{""reference"":""Shop_#DE-2797830""}",4,24.0,EUR,144,"[""air up de"",""ten-ace-gmbh.myshopify.com""]",DEDE1,DPSTD,depoma,Deutsche Post Mail,ce-de,304f976778aa89bf9335082085b3ccb5
